# <center>Structured Output</center>


In [23]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage

from pydantic import BaseModel
from PIL import Image
from io import BytesIO
import requests

from IPython.display import Markdown, display
from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
model_name = os.getenv("GEMINI_MODEL")


In [24]:
from pydantic import BaseModel

class PlanetInfo(BaseModel):
    name: str
    color: str
    distance_miles: int

In [29]:
model_client = OpenAIChatCompletionClient(
    model=model_name,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key,
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "gpt-4o",
        "structured_output": True
    },
    response_format=PlanetInfo
)

### Agent create

In [26]:
agent = AssistantAgent(
    name='planet_agent',
    model_client=model_client,
    system_message="You are a helpful assistant that provides information about planets. in the structure JSON"
)

In [34]:
async def test_structured_output():
    task = TextMessage(content = "Please provide information about Mars.",source='User')
    result = await agent.run(task=task)
    structured_response = result.messages[-1].content
    print(structured_response)
    return structured_response

resultJson = await test_structured_output()

{
  "color": "red",
  "distance_miles": 142000000,
  "name": "Mars"
}


In [ ]:
print("Result:")
print(resultJson)

if isinstance(resultJson, dict):
    json_string = json.dumps(resultJson)
    print("As JSON string:")
    print(json_string)
elif isinstance(resultJson, str):
    try:
        parsed_json = json.loads(resultJson)
        print("Parsed JSON:")
        print(parsed_json)
    except json.JSONDecodeError:
        print("Not a valid JSON string:", resultJson)

Result:
{
  "color": "red",
  "distance_miles": 142000000,
  "name": "Mars"
}
Parsed JSON:
{'color': 'red', 'distance_miles': 142000000, 'name': 'Mars'}
